In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from pyspark.sql import functions as F, DataFrame
import datetime as dt
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import pyspark.sql.types as t
from decimal import Decimal
from pyspark.sql.functions import regexp_replace

In [ ]:
from dataproc_sdk.dataproc_sdk_datiopysparksession.datiopysparksession import DatioPysparkSession
datioSparkSession = DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_datiopysparksession import datiopysparksession
dataproc = datiopysparksession.DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_schema.datioschema import DatioSchema
from dataproc_sdk.dataproc_sdk_datiofilesystem.datiofilesystem import DatioFileSystem

In [ ]:
# para evitar problemas de tipología de datos
spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

# para evitar problemas de particiones
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")

In [ ]:
# procesamiento de la cartera óptima en python
import pandas as pd
import numpy as np

# Configuracion

In [ ]:
fecha = date.today() # por defecto la fecha de hoy (se actualizará en el proceso con la más reciente)

In [ ]:
root_path = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/'
root_path

In [ ]:
# calculamos la fecha más reciente de la ruta tomando como campo de particion el pasado como parámetro
def last_partition (p_path:str, campo:str):
    
    datio_path = DatioFileSystem().get().qualify(p_path)
    fs = datio_path.fileSystem()
    path = datio_path.path()
    path_list = fs.listStatus(path)
    paths = [path.getPath().toString() for path in path_list] #listado de todos los paths de la ruta pasada
    
    l_fechas = [element.split(campo+'=')[1] for element in paths if campo in element] #listado de todas las fechas
    return max(l_fechas) # fecha mayor

# Cartera Titulizar

In [ ]:
fecha_ejecucion = last_partition (root_path, 'closing_date')
print('Fecha de ejecución del modelo titulización:', fecha_ejecucion)
root_pathc = root_path + 'closing_date=' + str(fecha_ejecucion)
# root_pathc

In [ ]:
path_out = root_pathc+'/cartera_titulizar'
path_out

In [ ]:
df_total = dataproc.read().parquet(path_out)

In [ ]:
df_total.show(5,False)

In [ ]:
# comprobamos que sólo son seleccionables las facilities con limites a nivel individual y portfolio >0 
df_total.where((F.col('selected')==1) & ((F.col('limit_individual')==0) | (F.col('limit_portfolio')==0))).count()

In [ ]:
# el máximo importe_optimo por facility nunca puede ser superior al limite_portfolio de la facility * portfolio size
df_total.groupBy().max('importe_optimo').show() #15.000.000 = 0.0075*2.000.000.000
print('máximo importe a nivel facility')

In [ ]:
# sorted(df_total.columns)

In [ ]:
df_total.groupBy('selected','motivo_exclusion'
                ).count().orderBy('selected','motivo_exclusion').show(truncate=False)

In [ ]:
df_total.groupBy('selected','motivo_exclusion','detalle_exclusion'
                ).count().orderBy('selected','motivo_exclusion','detalle_exclusion'
                                 ).show(500,truncate=False)

# Chequeos de aplicación

In [ ]:
facilities_id = ['830990','815808','848947']#['185065','783009']

In [ ]:
df_total.where(F.col('delta_file_id').isin(*facilities_id)).select('selected','delta_file_id','motivo_exclusion','detalle_exclusion'
                ).orderBy('candidata','selected','motivo_exclusion','detalle_exclusion').show(20,truncate=False)

## Aplicacion limites individuales

In [ ]:
# volumen de facilities según su limite individual
df_total.groupBy('limit_individual').count().show()

In [ ]:
cols_ind = ['delta_file_id',
        'limit_excluded_facilities','limit_rating_rga_esl','gf_ma_expanded_master_scale_id', #añado el campo que en este caso cumple limite
        'limit_rating_sp','limit_sts_rw_modelo','limit_sts_payment','limit_individual',
        'importe_susceptible','importe_titulizable','candidata',
        'motivo_exclusion','detalle_exclusion'] 

In [ ]:
df_total.where(F.col('delta_file_id').isin('185065','783009')).select(*cols_ind).show()

In [ ]:
df_total.where(F.col('delta_file_id').isin(facilities_id)).select(*cols_ind).show()

In [ ]:
df_total.where(F.col('candidata')==1).count() # facilities que pueden entrar en la cartera porque tienen importe titulizable

In [ ]:
# df_total.where(F.col('importe_titulizable')>0).count() # coincide con candidata=1

## Aplicacion limites portfolio
El máximo % del portfolio size por facility 
- todas están limitadas a un máximo de (2.000.000.000 * 0.0075 = 15.000.000), no puede entrar más importe a nivel facility

In [ ]:
# volumen de facilities CANDIDATAS según su limite portfolio
df_total.where(F.col('candidata')==1).groupBy('limit_portfolio').count().show()

In [ ]:
# volumen del TOTAL de facilities según su limite portfolio
# df_total.groupBy('limit_portfolio').count().show()

In [ ]:
cols_portf = ['delta_file_id',
        'limit_customer_subsector','limit_no_esg_linked','esg_linked_flag',# porque caso particular cumple limite
        'limit_customer_sector','g_asset_allocation_sector_desc',# porque caso particular cumple limite
        'limit_divisa','limit_customer_country','customer_country',# porque caso particular cumple limite
        'limit_sts_group','limit_financial_product',
        'limit_maturity_min','limit_non_ig','limit_group','limit_portfolio',
        'candidata','selected','importe_titulizable',]     
# por cada limite, está calculado el importe máximo de portfolio de ese limite (Ej. 'max_portfolio_size_customer_subsector',)

In [ ]:
df_total.where(F.col('delta_file_id').isin('185065','783009')).select(*cols_portf).show()

In [ ]:
cols_test = ['delta_file_id']
for k in 
cols_test.append('limit_'+k)
        cols_test.append('max_portfolio_size_'+k)
        cols_test.append('consumido_'+k)
        cols_test.append('importe_consumido_'+k)

In [ ]:
cols_consumos = ['delta_file_id',
                 'limit_group','consumido_group','disponible_group',
        'limit_customer_subsector','consumido_customer_sector','limit_no_esg_linked',
              'consumido_no_esg_linked','limit_customer_sector','consumido_customer_subsector',
        'limit_divisa','consumido_divisa','limit_customer_country','consumido_customer_country',
              'limit_sts_group','consumido_sts_group','limit_financial_product','consumido_financial_product',
        'limit_non_ig','consumido_non_ig','porcentaje_max_portfolio','porcentaje_portfolio_size','importe_max_facility',
                'selected','ranking_candidata','ranking_selected','importe_titulizable','importe_optimo','porcentaje_optimo']    

In [ ]:
df_total.where(F.col('delta_file_id').isin(facilities_id)).select(*cols_consumos).show()

In [ ]:
df_total.where(F.col('delta_file_id').isin('185065','783009')).select(*cols_consumos).show()

In [ ]:
df_total.select(*cols_consumos).orderBy('ranking_candidata').show()

### Non_ig

In [ ]:
df_total.groupBy().agg(F.max('consumido_non_ig')).show()

In [ ]:
df_total.groupBy('non_ig_flag','candidata').count().orderBy('non_ig_flag','candidata').show()

In [ ]:
df_total.where(F.col('consumido_non_ig')>0).select('ranking_candidata','selected','limit_non_ig','non_ig_flag',
                                                   'disponible_non_ig','consumido_non_ig',).orderBy('ranking_candidata').show(50,False)

## Datos cartera
Resultado final tras el algoritmo:
- importe óptimo: lo que el modelo recomienda tomar de la facility en base a los limites establecidos
- porcentaje óptimo: en % lo que se toma de la facility del total de su importe susceptible

In [ ]:
cols_cart = ['delta_file_id','selected',
             'ranking_candidata','ranking_selected',
             'importe_susceptible','importe_titulizable','importe_optimo',#'limit_portfolio',
            'porcentaje_optimo','porcentaje_portfolio_size',
            'limit_individual','limit_portfolio']
              
df_total.where(F.col('delta_file_id').isin('185065','783009')).select(*cols_cart).show()

# Por criterios

## Porcentaje - Importe optimo

In [ ]:
# chequeo % sobre el importe de la cartera -> si es 0 implica que importe optimo=0
df_total.where(F.col('porcentaje_portfolio_size')==0).select('importe_optimo').distinct().show()# debería ser posible solo 1 (importe=0)

In [ ]:
# chequeo % que se añade del importe suceptible a titulizar de la facility-> si es 0 implica que importe optimo=0
df_total.where(F.col('porcentaje_optimo')==0.0000).select('importe_optimo').distinct().show()# debería ser posible solo 1 (importe=0)

In [ ]:
df_total.where(F.col('importe_optimo')==0).select('porcentaje_optimo','porcentaje_portfolio_size').distinct().show()

In [ ]:
# contrasto el % a nivel facilitity, % a nivel portfolio_sixe, % que como máximo se puede titulizar del portfolio size
df_total.select('porcentaje_optimo','porcentaje_portfolio_size','porcentaje_min_portfolio').show()# debería ser posible solo 1 (importe=0)

In [ ]:
# chequeo sin porcentaje a añadir-> importe optimo=0
# df_total.where(F.col('porcentaje_min_portfolio')==0).select('importe_optimo').distinct().show()# debería ser posible solo 1 (importe=0)

## Rating

In [ ]:
# chequeo: limites 0% individual
df_total.where(F.col('g_lmscl_internal_ratg_type')=='CCC').select('limit_rating_sp').show(5,False)

In [ ]:
# limite por grupo (ok)
# df_total.select('limit_sts_group','limit_group').distinct().show()

In [ ]:
# chequeo: sin importe a añadir-> porcentaje optimo=0 (OK)
# df_total.where(F.col('importe_optimo')==0).select('porcentaje_optimo').distinct().show()

In [ ]:
# chequeo: limites 0% portfolio (OK)
# df_total.where(F.col('customer_country')=='Brazil').select('limit_customer_country').show(5,False) # 0.0

# Traza de exclusión
analizar el por qué de las facilities que se excluyen

In [ ]:
key = ['delta_file_id','delta_file_band_id','branch_id']

In [ ]:
id_grupo = 'G20220527162256' # la facility 13 ya se excluye porque se ha llegaod al 0.0075

In [ ]:
cols = [*key, 'g_holding_group_id','limit_group','consumido_group','importe_consumido_group',
        'porcentaje_max_portfolio','importe_titulizable','importe_optimo',
        'importe_susceptible','porcentaje_optimo',
        'selected','ranking_candidata','ranking_selected']

In [ ]:
df_total.where(F.col('g_holding_group_id')==id_grupo).select(*cols).orderBy('ranking_candidata').show(20,False)